In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
%matplotlib notebook
from Reinforcement_Learning import *
from Maze import *
from DrawMaze import *
import cv2
import matplotlib.pyplot as plt
import csv

In [2]:
PATH = 0 #道
WALL = 1 #壁
STR = 'S' #スタート
GOL = 'G' #ゴール
# mazeRead, mazeWrite, mazeDataFile = False, True, 'input/maze.csv'
mazeRead, mazeWrite, mazeDataFile = True, False, 'input/maze.csv'
if mazeRead:
    #迷路読み込み
    mazeData = []
    with open(mazeDataFile) as f:
        reader = csv.reader(f)
        for row in reader:
            mazeData.append(row)
    maze = Maze(mazeData, 1)
else:
    #迷路生成
    mWidth, mHeight = 20, 20
    maze = Maze([mWidth, mHeight], 0)
    maze.set_out_wall()
    maze.set_inner_wall_botaoshi()
    maze.set_start_goal()
    
maze.print_maze()
#迷路保存
if mazeWrite:
    maze.export_csv(mazeDataFile)
mWidth, mHeight = maze.get_width_height()
# 迷路データ maze.maze[x][y]

＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃
＃Ｓ　　　　　　　　　　　　＃　　　　　＃
＃　＃＃＃　＃＃＃＃＃＃＃＃＃　＃＃＃　＃
＃　＃　＃　　　　　＃　　　　　＃　＃　＃
＃　＃　＃＃＃＃＃　＃　＃　＃　＃　＃＃＃
＃　＃　　　＃　＃　＃　＃　＃　　　　　＃
＃　＃　＃　＃　＃　＃　＃＃＃　＃＃＃＃＃
＃　　　＃　＃　＃　　　　　＃　　　　　＃
＃＃＃　＃　＃　＃＃＃＃＃　＃＃＃　＃＃＃
＃　＃　＃　　　＃　　　＃　　　＃　＃　＃
＃　＃　＃　＃＃＃　＃＃＃　＃＃＃＃＃　＃
＃　　　＃　　　　　＃　＃　　　　　　　＃
＃＃＃　＃＃＃　＃＃＃　＃＃＃　＃＃＃＃＃
＃　　　＃　＃　＃　　　＃　＃　＃　　　＃
＃　＃＃＃　＃　＃　＃＃＃　＃　＃　＃　＃
＃　　　　　＃　　　　　＃　＃　＃　＃　＃
＃　＃　＃　＃＃＃＃＃　＃　＃　＃　＃　＃
＃　＃　＃　　　＃　＃　　　＃　　　＃　＃
＃　＃＃＃＃＃　＃　＃＃＃＃＃＃＃　＃＃＃
＃　　　　　＃　　　　　　　　　＃　　Ｇ＃
＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃


In [ ]:
# 強化学習
# q値更新
learningRate = 0.1 #学習係数α
discountRate = 0.9 #割引率γ
qUpdate = QLearning(learningRate, discountRate) # Q学習

#行動選択方法
ep = 0.3 #ε
actionSelect = EpGreedy(ep) #ε-greedy 

learning = ReinforcementLearning(mWidth, mHeight, qUpdate, actionSelect)

In [ ]:
#壁でないか確認
def is_not_wall(x, y):
    if maze.maze[x][y] == WALL:
        return False
    else:
        return True

In [ ]:
# 迷路描画機能
imgW = 800
imgH = 800
draw = DrawMaze(imgW, imgH, maze.maze)

In [ ]:
# 学習初期化
str_x, str_y = maze.get_start() #TODO Start位置の取得関数
gol_x, gol_y = maze.get_goal() #TODO Goal位置の取得関数
gol_reward = 30 #成功報酬
sx, sy = str_x, str_y
maxAction = 10000 # 最大行動数
maxResetAction = (mWidth + mHeight) * 30 # リセットになる行動回数
resetAction = 0

In [ ]:
for t in range(0, maxAction):
    # 次の行動を決定
    while True:
        nx, ny, action = learning.get_next_state(sx, sy)
        # 今回は次の行動が壁の時はキャンセルする
        if is_not_wall(nx, ny):
            break
    
    # Q値更新
    sq = learning.get_q_val(sx, sy, action)
    na, nq = learning.get_q_max_val(nx, ny)
    reward = gol_reward if (nx == gol_x and ny == gol_y) else 0
    p = [sq, nq, reward]
    learning.update_q_val(sx, sy, action, p)
    
    # 迷路描画
    draw.drawMaze(learning, gol_reward, sx, sy, nx, ny)
    sx, sy = nx, ny
    
    resetAction = resetAction + 1
    # リセット回数に達している、またはゴールならばスタートに戻る
    if (resetAction >= maxResetAction) or (sx == gol_x and sy == gol_y):
        resetAction = 0
        sx, sy = str_x, str_y    

In [ ]:
#結果出力(動画)
# encoder(for mp4)
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# output file name, encoder, fps, size(fit to image size)
video = cv2.VideoWriter('video.mp4',fourcc, 120.0, (imgW, imgH))
if not video.isOpened():
    print("can't be opened")
    sys.exit()
    
for i in range(0, maxAction):
    # hoge0000.png, hoge0001.png,..., hoge0090.png
    img = cv2.imread('output/images/img_' + str(i)+ '.jpg')

    # can't read image, escape
    if img is None:
        print("can't read")
        break

    # add
    video.write(img)
    print("read image " + str(i))

video.release()
print('written')